In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter

print(load_dotenv())

os.environ['OPENAI_API_KEY'] = os.getenv("OPEN_AI_API_KEY")

True


## **Custom agent**

**Define the functions**

In [3]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [4]:
abc(2, 3)

-2

In [5]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [6]:
abc_json = jsonschema(abc)
abc_json

{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [7]:
model_name = "gpt-3.5-turbo"

**Ask GPT**

In [8]:
client = OpenAI()

response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

In [9]:
response.choices[0].message.content

'To calculate \\(abc\\) between 2 and 3, we need to know the formula or equation that involves \\(a\\), \\(b\\), and \\(c\\). Could you please provide more details or the specific equation you are referring to?'

In [10]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

In [11]:
response

ChatCompletion(id='chatcmpl-ABYqUEz1RWgGqFIHp6iVWi7bUua4f', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"num1":2,"num2":3}', name='abc'), tool_calls=None))], created=1727318918, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=72, total_tokens=92, completion_tokens_details={'reasoning_tokens': 0}))

**Executing the function by extracting the info from the output of the model**

In [12]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{"num1":2,"num2":3}', name='abc')
{"num1":2,"num2":3}
<class 'str'>


In [13]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [14]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


## **Using Langchain**

In [15]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [17]:
abc.description

'Compute abc between two numbers'

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [19]:
tools = [abc]

llm_with_tools = llm.bind_tools(tools)

In [20]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

In [21]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_22aQbHVTNYFhgKCb5D9LaV5I',
   'function': {'arguments': '{"num1":2,"num2":3}', 'name': 'abc'},
   'type': 'function'}],
 'refusal': None}